1. Consider a Cox-Ross-Rubinstein binomial tree model with the expected value of relative stock return $(S(t) - S(t-1))/S(t-1)$ equal to $0.02$, and variance equal to $0.0216$. Suppose the up factor is $u = 1.2$

    Compute the value of the down factor $d$

    We got that
    $E\left[\frac{S(t) - S(t-1)}{S(t-1)}\right] = 0.02$, $Var\left[\frac{S(t) - S(t-1)}{S(t-1)}\right] = 0.0216$ and $p_u = \frac{e^{r\Delta t} - d}{u - d}$

    Under the model of CRR we know that the distribution underlying random variable corresponds to a bernulli distribution so:
    $$
    E\left[\frac{S(t) - S(t-1)}{S(t-1)}\right] = p (u-1) + (1-p)(d-1)
    $$

    for any random variable 
    $$
        Var(X) = E[X^2] - E[X]^2
    $$

    therefor is easy to show that
    $$
        E\left[\left(\frac{S(t) - S(t-1)}{S(t-1)}\right)^2\right] = p (u-1)^2 + (1-p)(d-1)^2
    $$

